In [1]:
import itertools
import pandas as pd
import utils
from collections import defaultdict, OrderedDict
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from scipy.sparse import hstack, vstack
import tensorflow as tf
from cnn_model import TextCNN
import data_helpers
import os
import time
import datetime

In [2]:
ordered_names = [u'study',
                 u'history',
                 u'comparison',
                 u'technique',
                 u'findings',
                 u'impression', 
                 u'signed by',
                 ]

--------
# Read data

In [3]:
filename = 'Data/upto1528.xlsx'
df_raw = pd.read_excel(open(filename, 'rb'))

In [4]:
# Data is stored in df
ps = utils.Parser()
ps.parse(df_raw)
df = ps.df
for idx, row in df['findings'].items():
    try:
        text, velos = utils.parse_findings(row)
        df.at[idx, 'findings'] = text
        for n, v in velos:
            df.at[0, n] = v
    except:
        pass
discardField = ['Report Text']
foo = [item for item in df.columns.tolist() if item not in ordered_names+discardField]
foo.sort()
CORE_COL = ordered_names + foo
df = df[CORE_COL]
df = pd.concat([df_raw[['Past', 'Present', 'Left', 'Right', 'Count']], df[CORE_COL]], axis=1)
# turn null to []
df = utils.null2empty(df, ['history', 'impression', 'comparison'])
print(df.shape)

(1527, 52)


/home/vzhao/miniconda3/envs/nlp/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


---
# Data Prep

In [5]:
# To predict PAST
to_predict = ['Past']
fields = [
#     'history', 
    'findings', 
#     'comparison', 
    'impression'
]
df_filtered = df[~df['Past'].isnull() & df['Past'] != 0].sample(frac=1, random_state=1)
df_filtered = df_filtered[to_predict + fields]

df_train = df_filtered.iloc[:1220]
y_train = np.array(df_train['Past'].astype(int))

df_test = df_filtered.iloc[1220:]
y_test = np.array(df_test['Past'].astype(int))

print(df_train.shape)
print(df_test.shape)

(1220, 3)
(307, 3)


---
# Tranditional ML

### TFIDF features

In [ ]:
ngram, min_count = 5, 5

obj = utils.Df2TFIDF()
obj.fit(df_train, ngram=ngram, min_count=min_count)
output_train = obj.transform(df_train)
output_test = obj.transform(df_test)
# concatenate sparse matrices of all fields
x_train = hstack([foo['bow_tfidf'] for foo in output_train.values()])
x_test = hstack([foo['bow_tfidf'] for foo in output_test.values()])

print(x_train.shape)
print(x_test.shape)

### Run classifier

In [ ]:
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
# svm
clf = LinearSVC(C=1, loss='squared_hinge')
#clf = SVC()
#clf = LogisticRegression(C=3)

clf.fit(x_train, y_train)
coef = clf.coef_.squeeze()

In [ ]:
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)
results = pd.concat([utils.my_classification_report(y_train, y_train_pred),
                     utils.my_classification_report(y_test, y_test_pred),
                     ], axis=1).transpose()
results.index = ['training', 'testing']
display(results)

In [ ]:
# features with the big weights
idx = np.absolute(coef).argsort()[::-1]
plt.plot(coef[idx])
plt.show()
idx2word_agg = pd.Series(obj.idx2word_concat)
display(idx2word_agg[idx[:20]])

In [ ]:
# output failed cases.
pd.set_option('display.max_colwidth', -1)
false_positive = (y_test_pred - y_test) > 0
display(df_test.loc[false_positive,:].applymap(utils.list2str))

---
---
# Neural Nets

### Data Prep

In [14]:
## Training data
x_train_text = utils.df2texts(df_train, df_train.columns[1:])
word2idx, idx2word = utils.ngram_vocab_processor(x_train_text, ngram=1, min_count=2)
x_train = np.array(utils.encode_texts(x_train_text, word2idx, maxlen=200))

y_train = df_train['Past'].values[:, None]
y_train = np.concatenate([(y_train + 1) / 2, (1 - y_train) / 2], axis=1).astype(np.int)

x_dev_text = utils.df2texts(df_test, df_train.columns[1:])
x_dev = np.array(utils.encode_texts(x_dev_text, word2idx, maxlen=x_train.shape[1]))

y_dev = df_test['Past'].values[:, None]
y_dev = np.concatenate([(y_dev + 1) / 2, (1 - y_dev) / 2], axis=1).astype(np.int)

### Run CNN

In [15]:
# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
# tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
# tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()

['-f',
 '/run/user/1000/jupyter/kernel-1f172ae7-ce5b-4024-88c3-8297b644efe2.json']

In [16]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(word2idx),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        # vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

Saved model checkpoint to /home/vzhao/Documents/Projects/Med-NLP/runs/1526175836/checkpoints/model-100

2018-05-12T21:44:00.313935: step 101, loss 0.413966, acc 0.859375
2018-05-12T21:44:00.338014: step 102, loss 0.45764, acc 0.828125
2018-05-12T21:44:00.368755: step 103, loss 1.23241, acc 0.734375
2018-05-12T21:44:00.398825: step 104, loss 0.725405, acc 0.8125
2018-05-12T21:44:00.420794: step 105, loss 0.782949, acc 0.734375
2018-05-12T21:44:00.445136: step 106, loss 0.793675, acc 0.75
2018-05-12T21:44:00.470234: step 107, loss 0.748904, acc 0.828125
2018-05-12T21:44:00.496400: step 108, loss 0.749364, acc 0.828125
2018-05-12T21:44:00.517343: step 109, loss 0.54769, acc 0.859375
2018-05-12T21:44:00.540517: step 110, loss 0.764875, acc 0.828125
2018-05-12T21:44:00.565067: step 111, loss 0.449173, acc 0.890625
2018-05-12T21:44:00.587564: step 112, loss 0.612851, acc 0.859375
2018-05-12T21:44:00.613487: step 113, loss 1.3654, acc 0.734375
2018-05-12T21:44:00.636520: step 114, loss 0.9622

2018-05-12T21:44:03.488269: step 227, loss 0.606824, acc 0.875
2018-05-12T21:44:03.513530: step 228, loss 0.460801, acc 0.859375
2018-05-12T21:44:03.534931: step 229, loss 0.319363, acc 0.890625
2018-05-12T21:44:03.558811: step 230, loss 0.332548, acc 0.890625
2018-05-12T21:44:03.582129: step 231, loss 0.60181, acc 0.828125
2018-05-12T21:44:03.610420: step 232, loss 0.586118, acc 0.8125
2018-05-12T21:44:03.634873: step 233, loss 0.574079, acc 0.828125
2018-05-12T21:44:03.661360: step 234, loss 0.340829, acc 0.890625
2018-05-12T21:44:03.686961: step 235, loss 0.24124, acc 0.90625
2018-05-12T21:44:03.710407: step 236, loss 0.325364, acc 0.90625
2018-05-12T21:44:03.738843: step 237, loss 0.322953, acc 0.875
2018-05-12T21:44:03.765637: step 238, loss 0.280945, acc 0.875
2018-05-12T21:44:03.789784: step 239, loss 0.395096, acc 0.828125
2018-05-12T21:44:03.803809: step 240, loss 0.0454161, acc 1
2018-05-12T21:44:03.835768: step 241, loss 0.385476, acc 0.875
2018-05-12T21:44:03.863793: step 2

2018-05-12T21:44:06.658443: step 355, loss 0.279127, acc 0.921875
2018-05-12T21:44:06.682989: step 356, loss 0.247541, acc 0.890625
2018-05-12T21:44:06.706514: step 357, loss 0.281829, acc 0.90625
2018-05-12T21:44:06.730094: step 358, loss 0.284573, acc 0.875
2018-05-12T21:44:06.755446: step 359, loss 0.506629, acc 0.859375
2018-05-12T21:44:06.764876: step 360, loss 0.00191314, acc 1
2018-05-12T21:44:06.788333: step 361, loss 0.410053, acc 0.875
2018-05-12T21:44:06.811051: step 362, loss 0.177228, acc 0.9375
2018-05-12T21:44:06.835335: step 363, loss 0.294089, acc 0.921875
2018-05-12T21:44:06.860240: step 364, loss 0.464902, acc 0.859375
2018-05-12T21:44:06.882962: step 365, loss 0.146344, acc 0.921875
2018-05-12T21:44:06.904413: step 366, loss 0.118707, acc 0.953125
2018-05-12T21:44:06.929204: step 367, loss 0.180679, acc 0.953125
2018-05-12T21:44:06.954441: step 368, loss 0.479068, acc 0.921875
2018-05-12T21:44:06.980608: step 369, loss 0.139969, acc 0.953125
2018-05-12T21:44:07.0052

2018-05-12T21:44:09.883103: step 486, loss 0.252708, acc 0.90625
2018-05-12T21:44:09.906878: step 487, loss 0.0754436, acc 0.984375
2018-05-12T21:44:09.935201: step 488, loss 0.246364, acc 0.90625
2018-05-12T21:44:09.961882: step 489, loss 0.0980451, acc 0.96875
2018-05-12T21:44:09.991868: step 490, loss 0.0718986, acc 0.984375
2018-05-12T21:44:10.017951: step 491, loss 0.106736, acc 0.953125
2018-05-12T21:44:10.041802: step 492, loss 0.247999, acc 0.953125
2018-05-12T21:44:10.067979: step 493, loss 0.111875, acc 0.953125
2018-05-12T21:44:10.093015: step 494, loss 0.206649, acc 0.953125
2018-05-12T21:44:10.115255: step 495, loss 0.239024, acc 0.90625
2018-05-12T21:44:10.146822: step 496, loss 0.16768, acc 0.9375
2018-05-12T21:44:10.172106: step 497, loss 0.13872, acc 0.984375
2018-05-12T21:44:10.202809: step 498, loss 0.264166, acc 0.875
2018-05-12T21:44:10.224062: step 499, loss 0.0938736, acc 0.953125
2018-05-12T21:44:10.238766: step 500, loss 0.0116485, acc 1

Evaluation:
2018-05-12

...

2018-05-12T21:45:21.750617: step 3406, loss 0.00563528, acc 1
2018-05-12T21:45:21.774403: step 3407, loss 0.00440686, acc 1
2018-05-12T21:45:21.800391: step 3408, loss 0.00485411, acc 1
2018-05-12T21:45:21.826311: step 3409, loss 0.00732973, acc 1
2018-05-12T21:45:21.850994: step 3410, loss 0.0015703, acc 1
2018-05-12T21:45:21.876519: step 3411, loss 0.0119872, acc 1
2018-05-12T21:45:21.900581: step 3412, loss 0.00721127, acc 1
2018-05-12T21:45:21.923702: step 3413, loss 0.00765001, acc 1
2018-05-12T21:45:21.948934: step 3414, loss 0.00595433, acc 1
2018-05-12T21:45:21.980605: step 3415, loss 0.0128854, acc 1
2018-05-12T21:45:22.005484: step 3416, loss 0.0680181, acc 0.984375
2018-05-12T21:45:22.033029: step 3417, loss 0.00358194, acc 1
2018-05-12T21:45:22.056129: step 3418, loss 0.0107966, acc 1
2018-05-12T21:45:22.080724: step 3419, loss 0.00606833, acc 1
2018-05-12T21:45:22.093711: step 3420, loss 0.00161931, acc 1
2018-05-12T21:45:22.119307: step 3421, loss 0.000720007, acc 1
2018-

2018-05-12T21:45:25.034016: step 3539, loss 0.00520557, acc 1
2018-05-12T21:45:25.059375: step 3540, loss 0.000119094, acc 1
2018-05-12T21:45:25.090161: step 3541, loss 0.0439716, acc 0.984375
2018-05-12T21:45:25.115164: step 3542, loss 0.00124501, acc 1
2018-05-12T21:45:25.139824: step 3543, loss 0.00467939, acc 1
2018-05-12T21:45:25.168656: step 3544, loss 0.00229348, acc 1
2018-05-12T21:45:25.191630: step 3545, loss 0.00114781, acc 1
2018-05-12T21:45:25.216672: step 3546, loss 0.00173535, acc 1
2018-05-12T21:45:25.246088: step 3547, loss 0.00386227, acc 1
2018-05-12T21:45:25.270899: step 3548, loss 0.0134327, acc 1
2018-05-12T21:45:25.295628: step 3549, loss 0.00334625, acc 1
2018-05-12T21:45:25.318993: step 3550, loss 0.00893004, acc 1
2018-05-12T21:45:25.342760: step 3551, loss 0.078228, acc 0.984375
2018-05-12T21:45:25.366387: step 3552, loss 0.00235051, acc 1
2018-05-12T21:45:25.388809: step 3553, loss 0.00093861, acc 1
2018-05-12T21:45:25.413745: step 3554, loss 0.00469771, acc

2018-05-12T21:45:28.221517: step 3670, loss 0.00537663, acc 1
2018-05-12T21:45:28.245987: step 3671, loss 0.00195609, acc 1
2018-05-12T21:45:28.269739: step 3672, loss 0.0021846, acc 1
2018-05-12T21:45:28.295769: step 3673, loss 0.00222635, acc 1
2018-05-12T21:45:28.320973: step 3674, loss 0.00442933, acc 1
2018-05-12T21:45:28.345780: step 3675, loss 0.00407161, acc 1
2018-05-12T21:45:28.369704: step 3676, loss 0.00545187, acc 1
2018-05-12T21:45:28.390778: step 3677, loss 0.00111945, acc 1
2018-05-12T21:45:28.414058: step 3678, loss 0.0361207, acc 0.984375
2018-05-12T21:45:28.439556: step 3679, loss 0.00128117, acc 1
2018-05-12T21:45:28.448945: step 3680, loss 4.0829e-06, acc 1
2018-05-12T21:45:28.471864: step 3681, loss 0.00107276, acc 1
2018-05-12T21:45:28.498477: step 3682, loss 0.00926879, acc 1
2018-05-12T21:45:28.520803: step 3683, loss 0.00047308, acc 1
2018-05-12T21:45:28.543001: step 3684, loss 0.00614211, acc 1
2018-05-12T21:45:28.565721: step 3685, loss 0.0605793, acc 0.9843

2018-05-12T21:45:31.493893: step 3803, loss 0.00204037, acc 1
2018-05-12T21:45:31.517374: step 3804, loss 0.011296, acc 0.984375
2018-05-12T21:45:31.541090: step 3805, loss 0.00519302, acc 1
2018-05-12T21:45:31.566831: step 3806, loss 0.00196818, acc 1
2018-05-12T21:45:31.590525: step 3807, loss 0.0032553, acc 1
2018-05-12T21:45:31.616084: step 3808, loss 0.00245009, acc 1
2018-05-12T21:45:31.641037: step 3809, loss 0.0746701, acc 0.984375
2018-05-12T21:45:31.664492: step 3810, loss 0.000518053, acc 1
2018-05-12T21:45:31.688232: step 3811, loss 0.0751283, acc 0.984375
2018-05-12T21:45:31.713673: step 3812, loss 0.00217808, acc 1
2018-05-12T21:45:31.736035: step 3813, loss 0.00309983, acc 1
2018-05-12T21:45:31.758702: step 3814, loss 0.0102373, acc 1
2018-05-12T21:45:31.784276: step 3815, loss 0.0051278, acc 1
2018-05-12T21:45:31.808991: step 3816, loss 0.0690353, acc 0.984375
2018-05-12T21:45:31.835441: step 3817, loss 0.00776436, acc 1
2018-05-12T21:45:31.860495: step 3818, loss 0.004

2018-05-12T21:45:34.761997: step 3937, loss 0.0524384, acc 0.984375
2018-05-12T21:45:34.785050: step 3938, loss 0.0495795, acc 0.984375
2018-05-12T21:45:34.807380: step 3939, loss 0.00314603, acc 1
2018-05-12T21:45:34.819509: step 3940, loss 0.000115309, acc 1
2018-05-12T21:45:34.842218: step 3941, loss 0.00183546, acc 1
2018-05-12T21:45:34.867233: step 3942, loss 0.00234094, acc 1
2018-05-12T21:45:34.891379: step 3943, loss 0.00351348, acc 1
2018-05-12T21:45:34.919728: step 3944, loss 0.00339666, acc 1
2018-05-12T21:45:34.942312: step 3945, loss 0.0715071, acc 0.984375
2018-05-12T21:45:34.965380: step 3946, loss 0.00330914, acc 1
2018-05-12T21:45:34.989824: step 3947, loss 0.00388412, acc 1
2018-05-12T21:45:35.020587: step 3948, loss 0.00979102, acc 1
2018-05-12T21:45:35.045915: step 3949, loss 0.0100441, acc 1
2018-05-12T21:45:35.069894: step 3950, loss 0.00168879, acc 1
2018-05-12T21:45:35.094908: step 3951, loss 0.0536209, acc 0.984375
2018-05-12T21:45:35.119404: step 3952, loss 0.

### Run RNN

### Sandbox

In [ ]:
a = [len(s) for s in cc]

In [ ]:
plt.hist(a)

---------

## Classification Model

## Use findings and impression to predict present and pass

In [ ]:
df['history'][1]

In [ ]:
#for PAST without inconsistency
fields = [
    'history', 
    'findings', 
    'comparison', 
    'impression',
]
foo = df[~df['Past'].isnull() & df['Count'] != 1].sample(frac=1)
df_train = foo.iloc[:1240]
y_train = np.array(df_train['Past'].astype(int))
df_test = foo.iloc[1240:]
y_test = np.array(df_test['Past'].astype(int))
obj = utils.Df2TFIDF()
obj.fit(df_train, fields, ngram=5, min_count=5)
output_train = obj.transform(df_train, fields)
output_test = obj.transform(df_test, fields)
# concatenate sparse matrices of all fields
from scipy.sparse import hstack
x_train = hstack([foo['bow_tfidf'] for foo in output_train.itervalues()])
# TODO (xiao)
x_test = hstack([foo['bow_tfidf'] for foo in output_test.itervalues()])
print x_train.shape
print x_test.shape

In [ ]:
#for PRESENT
fields = [
    'history', 
    'findings', 
    'comparison',
    'impression',
]
#'history', 'impression', 'findings', 'comparison'
foo = df[~df['Present'].isnull()].sample(frac=1)
foo = foo.loc[foo['Present']!=0, :]
df_train = foo.iloc[:1100]
y_train = np.array(df_train['Present'].astype(int))
df_test = foo.iloc[1100:]
y_test = np.array(df_test['Present'].astype(int))
obj = utils.Df2TFIDF()
obj.fit(df_train, fields, ngram=5, min_count=2)
output_train = obj.transform(df_train, fields)
output_test = obj.transform(df_test, fields)
# concatenate sparse matrices of all fields
x_train = hstack([foo['bow_tfidf'] for foo in output_train.itervalues()])
x_test = hstack([foo['bow_tfidf'] for foo in output_test.itervalues()])
print x_train.shape
print x_test.shape

In [ ]:
#for Left Grade
fields = ['history', 'findings', 'comparison','impression']
foo = df[~df['Left'].isnull() & df['Left'] != 0].sample(frac=1)
df_train = foo.iloc[:210]
y_train = np.array(df_train['Past'].astype(int))
df_test = foo.iloc[210:]
y_test = np.array(df_test['Past'].astype(int))
obj = utils.Df2TFIDF()
obj.fit(df_train, fields, ngram=5, min_count=5)
output_train = obj.transform(df_train, fields)
output_test = obj.transform(df_test, fields)

# concatenate sparse matrices of all fields
from scipy.sparse import hstack
x_train = hstack([foo['bow_tfidf'] for foo in output_train.itervalues()])
# TODO (xiao)
x_test = hstack([foo['bow_tfidf'] for foo in output_test.itervalues()])
print x_train.shape
print x_test.shape

In [ ]:
#for Left Grade
fields = ['history', 'findings', 'comparison','impression']
foo = df[~df['Right'].isnull() & df['Right'] != 0].sample(frac=1)
df_train = foo.iloc[:200]
y_train = np.array(df_train['Past'].astype(int))
df_test = foo.iloc[200:]
y_test = np.array(df_test['Past'].astype(int))
obj = utils.Df2TFIDF()
obj.fit(df_train, fields, ngram=5, min_count=5)
output_train = obj.transform(df_train, fields)
output_test = obj.transform(df_test, fields)

# concatenate sparse matrices of all fields
from scipy.sparse import hstack
x_train = hstack([foo['bow_tfidf'] for foo in output_train.itervalues()])
# TODO (xiao)
x_test = hstack([foo['bow_tfidf'] for foo in output_test.itervalues()])
print x_train.shape
print x_test.shape

### run classifier

In [ ]:
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
# svm
clf = LinearSVC(C=1, loss='squared_hinge')
#clf = SVC()
#clf = LogisticRegression(C=3)

clf.fit(x_train, y_train)
coef = clf.coef_.squeeze()

In [ ]:
x_train.shape

In [ ]:
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)
results = pd.concat([utils.my_classification_report(y_train, y_train_pred),
                     utils.my_classification_report(y_test, y_test_pred),
                     ], axis=1).transpose()
results.index = ['training', 'testing']
display(results)

In [ ]:
# features with the big weights
idx = np.absolute(coef).argsort()[::-1]
plt.plot(coef[idx])
plt.show()
idx2word_agg = pd.Series(obj.idx2word_concat)
display(idx2word_agg[idx[:20]])

In [ ]:
# output failed cases.
pd.set_option('display.max_colwidth', -1)
fields = [
    'Past',
    'history', 
    'findings', 
    'comparison', 
    'impression'
]
false_positive = (y_test_pred - y_test) > 0
display(df_test.loc[false_positive,fields].applymap(utils.list2str))

---
# Semi - Supervised Learning

## Prepare data

### Labeled Data

In [ ]:
filename = 'Data/upto1528.xlsx'
df_raw = pd.read_excel(open(filename, 'r'))
# get data frame
ps = utils.Parser()
ps.parser(df_raw)
df = ps.df
for idx, row in df['findings'].iteritems():
    try:
        text, velos = utils.parse_findings(row)
        df.set_value(idx, 'findings', text)
        for n, v in velos:
            df.set_value(idx, n, v)
    except:
        pass
discardField = ['Report Text']
foo = [item for item in df.columns.tolist() if item not in ordered_names+discardField]
foo.sort()
CORE_COL = ordered_names + foo
df = df[CORE_COL]
df_labeled = pd.concat([df_raw[['Past', 'Present', 'Left', 'Right', 'Count']], df[CORE_COL]], axis=1)

### Unlabeled Data

In [ ]:
filename = 'Data/2011-2016.xlsx'
df_raw = pd.read_excel(open(filename, 'r'))
# get data frame
ps = utils.Parser()
ps.parser(df_raw)
df = ps.df
for idx, row in df['findings'].iteritems():
    try:
        text, velos = utils.parse_findings(row)
        df.set_value(idx, 'findings', text)
        for n, v in velos:
            df.set_value(idx, n, v)
    except:
        pass
discardField = ['Report Text']
foo = [item for item in df.columns.tolist() if item not in ordered_names+discardField]
foo.sort()
CORE_COL = ordered_names + foo
df_unlabeled = df[CORE_COL]
# df = pd.concat([df_raw[['Past', 'Present', 'Left', 'Right', 'Count']], df[CORE_COL]], axis=1)

----

In [ ]:
#for PAST
fields = [
    'history', 
    'findings', 
    'comparison', 
    'impression'
]
# labeled data
df_labeled = utils.null2empty(df_labeled, fields)
foo = df_labeled[~df_labeled['Past'].isnull() & df_labeled['Past'] != 0].sample(frac=1, random_state=1)
df_train_labeled = foo.iloc[:1220]
y_train_labeled = np.array(df_train_labeled['Past'].astype(int))
y_train_labeled[y_train_labeled==-1] = 0 # turn -1 to 0
df_test = foo.iloc[1220:]
y_test = np.array(df_test['Past'].astype(int))
y_test[y_test==-1] = 0 # turn -1 to 0
obj = utils.Df2TFIDF()
obj.fit(df_train_labeled, fields, ngram=5, min_count=5)
output_train_labeled = obj.transform(df_train_labeled, fields)
output_test = obj.transform(df_test, fields)
# concatenate sparse matrices of all fields
x_train_labeled = hstack([foo['bow_tfidf'] for foo in output_train_labeled.itervalues()])
x_test = hstack([foo['bow_tfidf'] for foo in output_test.itervalues()])
print x_train_labeled.shape
print x_test.shape
# unlabeled data
df_unlabeled = utils.null2empty(df_unlabeled, fields)
# obj = utils.Df2TFIDF()
# obj.fit(df_train_unlabeled, fields, ngram=5, min_count=5)
output_train_unlabeled = obj.transform(df_unlabeled, fields)
x_train_unlabeled = hstack([foo['bow_tfidf'] for foo in output_train_unlabeled.itervalues()])
print x_train_unlabeled.shape

In [ ]:
x_train = vstack([x_train_labeled, x_train_unlabeled])
y_train = np.append(y_train_labeled, np.ones(x_train_unlabeled.shape[0], dtype=int) * -1)

In [ ]:
from sklearn.semi_supervised import LabelPropagation
semi_clf = LabelPropagation(kernel='rbf', gamma=20, n_jobs=-1)
semi_clf.fit(x_train.toarray(), y_train)

In [ ]:
y_train_pred = semi_clf.predict(x_train_labeled.toarray())
y_test_pred = semi_clf.predict(x_test.toarray())
results = pd.concat([utils.my_classification_report(y_train_labeled, y_train_pred),
                     utils.my_classification_report(y_test, y_test_pred),
                     ], axis=1).transpose()
results.index = ['training', 'testing']
display(results)